In [1]:
# import standard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic lines
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#import my scripts
import sys, os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    scripts_path = os.path.join(module_path, "scripts")
    if scripts_path not in sys.path:
        sys.path.append(scripts_path)
    display(sys.path)
   
from data import get_data
from preprocessing import preprocess, add_new_columns

['/home/louis/code/lewagon/data/04-Decision-Science/01-Project-Setup/context-and-setup',
 '/home/louis/code/JammyNinja/Sandbox/chess_analysis/explore/notebooks',
 '/home/louis/.pyenv/versions/3.10.6/lib/python310.zip',
 '/home/louis/.pyenv/versions/3.10.6/lib/python3.10',
 '/home/louis/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload',
 '',
 '/home/louis/.pyenv/versions/3.10.6/envs/chess_analysis/lib/python3.10/site-packages',
 '/home/louis/code/JammyNinja/Sandbox/chess_analysis/explore/scripts']

In [3]:
fname = "all_games_2023-01_to_2025-01_all_new_cols.csv"
path = os.path.join("..", "data", fname)
raw_df = pd.read_csv(path)
raw_df.head(3)

,date,start_time,end_time,time_class,time_control,rated,rules,url,moves,opening_code,...,winner_pawns_count,loser_pawns_count,winner_pieces_only_count,loser_pieces_only_count,pieces_count_diff,pawns_count_diff,winner_material_ttl,loser_material_ttl,winner_pcs,loser_pcs
0,2023-01-07,2023-01-07 08:17:06,2023-01-07 14:35:53,daily,1/604800,True,chess,https://www.chess.com/game/daily/437377841,1. e4 {[%clk 158:31:38] 1... c6 {[%clk 163:21:...,B10,...,6,6,5,4,1,0,31,22,"['r', 'k', 'r', 'p', 'p', 'p', 'p', 'p', 'q', ...","['P', 'P', 'B', 'P', 'P', 'P', 'P', 'K', 'R', ..."
1,2023-01-14,2023-01-14 11:34:48,2023-01-14 15:09:28,daily,1/604800,True,chess,https://www.chess.com/game/daily/448245095,1. e4 {[%clk 167:35:57] 1... c5 {[%clk 165:23:...,B22,...,5,6,6,6,0,-1,33,34,"['r', 'q', 'r', 'b', 'b', 'p', 'k', 'p', 'p', ...","['B', 'Q', 'P', 'N', 'P', 'P', 'P', 'P', 'P', ..."
2,2023-01-30,2023-01-30 17:32:36,2023-01-30 07:50:02,daily,1/604800,True,chess,https://www.chess.com/game/daily/436095119,1. e4 {[%clk 167:38:55] 1... c6 {[%clk 163:21:...,B10,...,7,7,3,3,0,0,20,20,"['r', 'r', 'k', 'n', 'p', 'p', 'p', 'p', 'p', ...","['P', 'P', 'P', 'R', 'P', 'P', 'N', 'P', 'P', ..."


# which piece do you move the most?

In [19]:
moves_cols = list(raw_df.columns[raw_df.columns.str.contains('moves')])
moves_cols.append("user_colour")

In [20]:
raw_df.columns

Index(['date', 'start_time', 'end_time', 'time_class', 'time_control', 'rated',
       'rules', 'url', 'moves', 'opening_code', 'opening_name',
       'white_username', 'black_username', 'white_clock', 'black_clock',
       'white_rating', 'black_rating', 'result', 'white_result',
       'black_result', 'winner', 'user_win', 'user_colour', 'user_rating',
       'user_rating_diff', 'higher_rated_colour', 'winner_rating_diff',
       'move_numbers', 'white_moves', 'black_moves', 'w_cstl_side',
       'w_cstl_move', 'b_cstl_side', 'b_cstl_move', 'user_castled',
       'opp_castled', 'final_position_fen', 'winner_ttl_pieces_count',
       'loser_ttl_pieces_count', 'winner_pawns_count', 'loser_pawns_count',
       'winner_pieces_only_count', 'loser_pieces_only_count',
       'pieces_count_diff', 'pawns_count_diff', 'winner_material_ttl',
       'loser_material_ttl', 'winner_pcs', 'loser_pcs'],
      dtype='object')

In [21]:
raw_df[moves_cols]

,moves,white_moves,black_moves,user_colour
0,1. e4 {[%clk 158:31:38] 1... c6 {[%clk 163:21:...,"['e4', 'Bc4', 'exd5', 'Bb3', 'd4', 'Qe2', 'Qd3...","['c6', 'd5', 'cxd5', 'Nf6', 'Nc6', 'Nxd4', 'e5...",black
1,1. e4 {[%clk 167:35:57] 1... c5 {[%clk 165:23:...,"['e4', 'c3', 'd4', 'dxc5', 'cxd6', 'Bb5', 'Bf4...","['c5', 'Nc6', 'd6', 'b6', 'exd6', 'Bb7', 'Nf6'...",white
2,1. e4 {[%clk 167:38:55] 1... c6 {[%clk 163:21:...,"['e4', 'Nf3', 'exd5', 'b3', 'h3', 'Qxf3', 'Bb5...","['c6', 'd5', 'cxd5', 'Bg4', 'Bxf3', 'e6', 'Nc6...",black
3,1. Nf3 {[%clk 0:04:54.9] 1... d5 {[%clk 0:04:5...,"['Nf3', 'c4', 'e4', 'a4', 'axb5', 'Nc3', 'b3',...","['d5', 'dxc4', 'b5', 'Bd7', 'Bxb5', 'Ba6', 'cx...",black
4,1. c4 {[%clk 164:52:18] 1... e5 {[%clk 125:36:...,"['c4', 'Nc3', 'f3']","['e5', 'Bc5', nan]",black
...,...,...,...,...
5919,1. e4 {[%clk 0:01:00] 1... e5 {[%clk 0:01:00] ...,"['e4', 'Nc3', 'Bc4', 'Qf3', 'Nh3', 'd3', 'Bg5'...","['e5', 'Nc6', 'Bc5', 'Nf6', 'O-O', 'd6', 'Be6'...",white
5920,1. f4 {[%clk 0:01:00] 1... d5 {[%clk 0:01:00] ...,"['f4', 'e4', 'd3', 'Nc3', 'Bd2', 'Nxe4', 'Qxd2...","['d5', 'dxe4', 'e6', 'Bb4', 'Nc6', 'Bxd2+', 'N...",black
5921,1. e4 {[%clk 0:01:00] 1... d5 {[%clk 0:01:00] ...,"['e4', 'exd5', 'd4', 'Be3', 'c4', 'Nc3', 'Bd3'...","['d5', 'Qxd5', 'Qd8', 'Nf6', 'Bf5', 'e6', 'Bg6...",black
5922,1. e4 {[%clk 0:01:00] 1... d6 {[%clk 0:01:00] ...,"['e4', 'd4', 'Nc3', 'Be3', 'Bd3', 'Nge2', 'h4'...","['d6', 'e6', 'Nd7', 'Be7', 'Ngf6', 'O-O', 'e5'...",white


In [25]:
df = raw_df.copy()

def my_func(df):
    return df['white_moves'] if df['user_colour'] == "white" else df['black_moves']
    
df['user_moves'] = df.apply(my_func, axis=1)

In [26]:
df['user_moves']

0       ['c6', 'd5', 'cxd5', 'Nf6', 'Nc6', 'Nxd4', 'e5...
1       ['e4', 'c3', 'd4', 'dxc5', 'cxd6', 'Bb5', 'Bf4...
2       ['c6', 'd5', 'cxd5', 'Bg4', 'Bxf3', 'e6', 'Nc6...
3       ['d5', 'dxc4', 'b5', 'Bd7', 'Bxb5', 'Ba6', 'cx...
4                                      ['e5', 'Bc5', nan]
                              ...                        
5919    ['e4', 'Nc3', 'Bc4', 'Qf3', 'Nh3', 'd3', 'Bg5'...
5920    ['d5', 'dxe4', 'e6', 'Bb4', 'Nc6', 'Bxd2+', 'N...
5921    ['d5', 'Qxd5', 'Qd8', 'Nf6', 'Bf5', 'e6', 'Bg6...
5922    ['e4', 'd4', 'Nc3', 'Be3', 'Bd3', 'Nge2', 'h4'...
5923    ['e4', 'Nc3', 'e5', 'd4', 'Nf3', 'Be3', 'Bxd4'...
Name: user_moves, Length: 5924, dtype: object

In [28]:
piece_move_dict = {
    "N" : "knight",
    "K" : "king",
    "Q" : "queen",
    "B" : "bishop",
    "R" : "rook",
    "" : 'pawn'
}
piece_move_dict[""]

'pawn'

In [45]:
import json

In [ ]:
def process_moves(moves):
    srs_of_list_of_moves = moves.str.split(',')
    

In [58]:
list_of_moves = df['user_moves'].str.split(',')#.apply(lambda x : x.strip("'"))
for move in list_of_moves[0]:
    print(move.strip("'")[1:])

'c6
'd5
'cxd5
'Nf6
'Nc6
'Nxd4
'e5
'Be6
'Qa5+
'Ne4
'Nxd2
'N2xb3
'Nc2+
'Nxe3
'Qb5+
'd4
'Bb4+
'Rc8+
'Bf5+
'Qa6#']


In [29]:
def move_piece(move):
    """
        converts a move (eg: 'Nxf5') to which piece was moved
    """

    if move[0].isupper():
        return move[0]
test_moves = df['user_moves'].iloc[0]
test_moves.apply(move_piece)

AttributeError: 'str' object has no attribute 'apply'